In [1]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Constructing a LangGraphAgent

| | |
|-|-|
|Author(s) | [Pablo Gaeta](https://github.com/pablofgaeta) |

## Overview

This notebook demonstrates the necessary code to transform any LangGraph graph into a `LangGraphAgent` using the `concierge.langgraph_server` module.

This is a useful interactive way to develop agents before deploying them on a server.

## Import dependencies

In [2]:
import asyncio
from typing import TypedDict
import uuid

from concierge.langgraph_server import langgraph_agent, schemas
from langgraph import graph, types

## Define graph state schema and simple graph implementation

In [3]:
# State schema


class State(TypedDict, total=False):
    id: str


# Node functions


async def set_id(state: State):
    new_id = state.get("id")
    assert new_id is not None, "must set ID"

    await asyncio.sleep(1)
    return types.Command(update=State(id=new_id), goto="REVERSE_ID")


async def reverse_id(state: State):
    new_id = state.get("id")
    assert new_id is not None, "ID must be set before reversing"

    await asyncio.sleep(1)
    return types.Command(update=State(id=new_id[::-1]))


# Graph

state_graph = graph.StateGraph(state_schema=State)
state_graph.add_node("SET_ID", set_id)
state_graph.add_node("REVERSE_ID", reverse_id)
state_graph.set_entry_point("SET_ID");

## Build a _stateful_ agent

In [4]:
basic_agent = langgraph_agent.LangGraphAgent(
    state_graph=state_graph,
    checkpointer_config=schemas.MemoryBackendConfig(),
)

**Test**: Run a query in debug stream mode

In [5]:
async for chunk in basic_agent.stream(
    input=State(id="hello"),
    config={"configurable": {"thread_id": uuid.uuid4().hex}},
    stream_mode="debug",
):
    print(chunk)

('debug', {'type': 'checkpoint', 'timestamp': '2025-03-31T17:23:05.184735+00:00', 'step': -1, 'payload': {'config': {'tags': [], 'metadata': {'thread_id': 'b91410cd74b34b6092deebd0bcbe7ef6'}, 'callbacks': None, 'recursion_limit': 25, 'configurable': {'checkpoint_ns': '', 'agent_config': {}, 'thread_id': 'b91410cd74b34b6092deebd0bcbe7ef6', 'checkpoint_id': '1f00e54c-e6e1-62e8-bfff-763f424dc8bb'}}, 'parent_config': None, 'values': {}, 'metadata': {'source': 'input', 'writes': {'__start__': {'id': 'hello'}}, 'thread_id': 'b91410cd74b34b6092deebd0bcbe7ef6', 'step': -1, 'parents': {}}, 'next': ['__start__'], 'tasks': [{'id': '193221f5-c915-43c9-5392-21797217efed', 'name': '__start__', 'interrupts': (), 'state': None}]}})
('debug', {'type': 'checkpoint', 'timestamp': '2025-03-31T17:23:05.185977+00:00', 'step': 0, 'payload': {'config': {'tags': [], 'metadata': {'thread_id': 'b91410cd74b34b6092deebd0bcbe7ef6'}, 'callbacks': None, 'recursion_limit': 25, 'configurable': {'checkpoint_ns': '', 'ag

**Test**: Run a query in values mode (full state returned each time)

In [6]:
async for chunk in basic_agent.stream(
    input=State(id="hello"),
    config={"configurable": {"thread_id": uuid.uuid4().hex}},
    stream_mode="values",
):
    print("complete state", "=", chunk)

complete state = ('values', {'id': 'hello'})
complete state = ('values', {'id': 'hello'})
complete state = ('values', {'id': 'olleh'})


**Test**: Run a query in updates mode (only state updates returned by each node)

In [7]:
async for chunk in basic_agent.stream(
    input=State(id="hello"),
    config={"configurable": {"thread_id": uuid.uuid4().hex}},
    stream_mode="updates",
):
    print("updates", "->", chunk)

updates -> ('updates', {'SET_ID': {'id': 'hello'}})
updates -> ('updates', {'REVERSE_ID': {'id': 'olleh'}})


**Test**: Run a query with a list of stream modes (yields stream mode and chunk data)

In [8]:
async for stream_mode, chunk in basic_agent.stream(
    input=State(id="hello"),
    config={"configurable": {"thread_id": uuid.uuid4().hex}},
    stream_mode=["updates"],
):
    print(stream_mode, "->", chunk)

updates -> {'SET_ID': {'id': 'hello'}}
updates -> {'REVERSE_ID': {'id': 'olleh'}}


**Test**: Attempt a stateless query, fails since checkpointer requires thread ID. To run stateless queries, must build a stateless agent like the example in the next section.

In [9]:
async for stream_mode, chunk in basic_agent.stream(
    input=State(id="hello"),
    config={"configurable": {"thread_id": uuid.uuid4().hex}},
    stream_mode=["updates"],
):
    print(stream_mode, "->", chunk)

updates -> {'SET_ID': {'id': 'hello'}}
updates -> {'REVERSE_ID': {'id': 'olleh'}}


## Build a _stateless_ agent

In [10]:
stateless_basic_agent = langgraph_agent.LangGraphAgent(
    state_graph=state_graph,
    checkpointer_config=None,  # no checkpointer means it will only work with stateless requests
)

**Test**: Run stateless run without threads

In [11]:
async for chunk in stateless_basic_agent.stream(
    input=State(id="hello"),
    stream_mode="updates",
):
    print("complete state", "=", chunk)

complete state = ('updates', {'SET_ID': {'id': 'hello'}})
complete state = ('updates', {'REVERSE_ID': {'id': 'olleh'}})
